## 146. LRU Cache

In [25]:
class DoubleLinkedNode:
    def __init__(self, key=0, val=0, pre=None, nxt=None):
        self.key = key
        self.val = val
        self.pre = pre
        self.nxt = nxt

class LRUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cur_capacity = 0
        self.memo = dict()
        self.head = DoubleLinkedNode()
        self.tail = DoubleLinkedNode()
        self.head.nxt = self.tail
        self.tail.pre = self.head

    def delete_node(self, node):
        node.pre.nxt = node.nxt
        node.nxt.pre = node.pre

    def add_node(self, node):
        node.nxt = self.head.nxt
        node.pre = self.head

        self.head.nxt.pre = node
        self.head.nxt = node

    def move_to_head(self, node):
        self.delete_node(node)
        self.add_node(node)


    def get(self, key: int) -> int:
        node = self.memo.get(key)
        if not node: return -1
        self.move_to_head(node)
        return node.val

    def put(self, key: int, value: int) -> None:
        node = self.memo.get(key)
        if node:
            node.val = value
            self.move_to_head(node)
        else:
            if self.cur_capacity == self.capacity:
                node = self.tail.pre
                if not node:
                    print(key, value)
                self.delete_node(node)
                del self.memo[node.key]
                self.cur_capacity -= 1
            node = DoubleLinkedNode(key, value)
            self.memo[key] = node
            self.add_node(node)
            self.cur_capacity += 1


In [26]:
lru = LRUCache(2)

lru.put(1, 1)
lru.put(2, 2)
lru.get(1)

insert 0 2 1 1
add (1, 1), cur capacity is 1
insert 1 2 2 2
add (2, 2), cur capacity is 2
here
get (1, 1), cur capacity is 2


1

We can also use the builtin OrderedDict

An OrderedDict is a dictionary subclass that remembers the order that keys were first inserted.
The only difference between dict() and OrderedDict() is that:

OrderedDict preserves the order in which the keys are inserted. A regular
 dict doesn’t track the insertion order, and iterating it gives the values in an
 arbitrary order. By contrast, the order the items are inserted is remembered by OrderedDict.



In [28]:
from collections import OrderedDict

In [34]:
od = OrderedDict()
od['a'] = 1
od['b'] = 2
od['c'] = 3

In [30]:
for key in od:
    print(key, od[key])

a 1
b 2
c 3


In [36]:
od.popitem(False)

('a', 1)

In [32]:
for key in od:
    print(key, od[key])

a 1
c 3


In [33]:
od.pop

<function OrderedDict.pop>

In [37]:
# the ordereddict version
class LRUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.od = OrderedDict()

    def get(self, key: int) -> int:
        val = self.od.get(key)
        if not val: return -1
        self.od.move_to_end(key)
        return val

    def put(self, key: int, value: int) -> None:
        val = self.od.get(key)
        if val:
            self.od[key] = value
            self.od.move_to_end(key)
        else:
            if len(self.od) == self.capacity:
                # pop the first one.
                self.od.popitem(False)
            self.od[key] = value

In [39]:
lru = LRUCache(2)

lru.put(1, 1)
lru.put(2, 2)
lru.get(2)


2